#### Solution 생성 및 Versioning
솔루선 생성 및 하이퍼 파라미터 조정

In [18]:
import boto3
from botocore.exceptions import ClientError
import json
import time


personalize = boto3.client("personalize")

#### 솔루션 생성

In [2]:
# recipe (알고리즘) 선택
recipe_arn = "arn:aws:personalize:::recipe/aws-user-personalization"

In [4]:
# 이미 dataset group이 생성되어 있을 때, 조회
# dataset_group_name = "osungmart-dataset-groupmwomk"
dataset_group_name = "osungmart-dataset-group20230204"

list_dataset_group_resp = personalize.list_dataset_groups(
  maxResults=10
)

for dataset_grp in list_dataset_group_resp['datasetGroups']:
  if dataset_grp['name'] == dataset_group_name:
    dataset_group_arn = dataset_grp['datasetGroupArn']
    print("dataset_group_arn: {}".format(dataset_grp['datasetGroupArn']) )

dataset_group_arn: arn:aws:personalize:ap-northeast-2:491818659652:dataset-group/osungmart-dataset-group20230204


In [5]:
# create solution
solution_name = "osugmart-user-personalize-solution-HPO"
try: 
  create_solution_resp = personalize.create_solution(
    name = solution_name,
    datasetGroupArn = dataset_group_arn,
    recipeArn = recipe_arn,
    performHPO=True
  )
  solution_arn = create_solution_resp['solutionArn']
  print(json.dumps(create_solution_resp, indent=2))
except ClientError as e:
  print(e)

{
  "solutionArn": "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO",
  "ResponseMetadata": {
    "RequestId": "84d36286-6df1-4467-97ac-b24ccc62b139",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Sat, 04 Feb 2023 13:02:14 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "113",
      "connection": "keep-alive",
      "x-amzn-requestid": "84d36286-6df1-4467-97ac-b24ccc62b139"
    },
    "RetryAttempts": 0
  }
}


In [19]:
solution_arn = "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO"

In [20]:
# Create Solution Version: training and retraining
try:
  create_solution_version_resp = personalize.create_solution_version(
    solutionArn = solution_arn
  )
  solution_version_arn = create_solution_version_resp['solutionVersionArn']
  print(json.dumps(create_solution_version_resp, indent=2))
except ClientError as e:
  print(e)


{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO/afda062c",
  "ResponseMetadata": {
    "RequestId": "d0158eee-6126-4339-9283-b46b692031e4",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 07 Feb 2023 06:43:53 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "129",
      "connection": "keep-alive",
      "x-amzn-requestid": "d0158eee-6126-4339-9283-b46b692031e4"
    },
    "RetryAttempts": 0
  }
}


In [7]:
status = None
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_solution_version_response = personalize.describe_solution_version(
        solutionVersionArn = solution_version_arn
    )
    status = describe_solution_version_response["solutionVersion"]["status"]
    print("SolutionVersion: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS
SolutionVersion: CREATE IN_PROGRESS


KeyboardInterrupt: 

#### 성능 평가
AWS User Personalization metrics

In [10]:
personalize_runtime = boto3.client('personalize-runtime')
personalize_events = boto3.client("personalize-events")

In [13]:
solution_version_arn = "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO/c80bae27"

In [17]:
get_solution_metrics_response = personalize.get_solution_metrics(
    solutionVersionArn = solution_version_arn
)
print(json.dumps(get_solution_metrics_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:ap-northeast-2:491818659652:solution/osugmart-user-personalize-solution-HPO/c80bae27",
  "metrics": {
    "coverage": 0.0014,
    "mean_reciprocal_rank_at_25": 0.084,
    "normalized_discounted_cumulative_gain_at_10": 0.1086,
    "normalized_discounted_cumulative_gain_at_25": 0.1262,
    "normalized_discounted_cumulative_gain_at_5": 0.0974,
    "precision_at_10": 0.0152,
    "precision_at_25": 0.0089,
    "precision_at_5": 0.024
  },
  "ResponseMetadata": {
    "RequestId": "9ef784c0-0933-457f-9ef3-e630965fa790",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "date": "Tue, 07 Feb 2023 06:29:06 GMT",
      "content-type": "application/x-amz-json-1.1",
      "content-length": "425",
      "connection": "keep-alive",
      "x-amzn-requestid": "9ef784c0-0933-457f-9ef3-e630965fa790"
    },
    "RetryAttempts": 0
  }
}
